In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import math
import random
import os
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy

random.seed(30)

Se exportan los datos con que se entreno el modelo, dejar así porque estos datos tienen las clases.

In [3]:
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/data.zip", "/content/data.zip")
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/Comerciales.csv", "/content/Comerciales.csv")
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/2021_10_27_09_F10020.jpg  
  inflating: data/2021_10_27_09_F10080.jpg  
  inflating: data/2021_10_27_09_F10140.jpg  
  inflating: data/2021_10_27_09_F1020.jpg  
  inflating: data/2021_10_27_09_F10200.jpg  
  inflating: data/2021_10_27_09_F10260.jpg  
  inflating: data/2021_10_27_09_F10320.jpg  
  inflating: data/2021_10_27_09_F10380.jpg  
  inflating: data/2021_10_27_09_F10440.jpg  
  inflating: data/2021_10_27_09_F10500.jpg  
  inflating: data/2021_10_27_09_F10560.jpg  
  inflating: data/2021_10_27_09_F10620.jpg  
  inflating: data/2021_10_27_09_F10680.jpg  
  inflating: data/2021_10_27_09_F10740.jpg  
  inflating: data/2021_10_27_09_F1080.jpg  
  inflating: data/2021_10_27_09_F10800.jpg  
  inflating: data/2021_10_27_09_F10860.jpg  
  inflating: data/2021_10_27_09_F10920.jpg  
  inflating: data/2021_10_27_09_F10980.jpg  
  inflating: data/2021_10_27_09_F11040.jpg  
  inflating: data/2021_10_27_09_F11100.jpg  
  inflating: data/2

El dataset de los datos con que se entreno el modelo.

In [4]:
df = pd.read_csv('Comerciales.csv', header=0, sep=";")

In [5]:
df

,Nombre Final,Clase
0,2021_10_27_09_F60,Noticias
1,2021_10_27_09_F120,Noticias
2,2021_10_27_09_F180,Noticias
3,2021_10_27_09_F240,Noticias
4,2021_10_27_09_F300,Noticias
...,...,...
3559,2021_10_27_13_F120720,Comercial
3560,2021_10_27_13_F120780,Comercial
3561,2021_10_27_13_F120840,Comercial
3562,2021_10_27_13_F120900,Comercial


In [6]:
classesFinal = df["Clase"].unique()
#classesFinal = [cl.replace(' ', '_') for cl in classes]
print(classesFinal)

['Noticias' 'Comercial']


Se necesita de una sola carpeta, aquella que es de test. Esta carpeta se dejará vacía a excepción de una imagen en cada carpeta de cada clase. (Si no se hace esto se generá un error después)

In [7]:
try:
  os.mkdir('dataset')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")


try:
  os.mkdir('dataset/test')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")



for cl in classesFinal:

  
  try:
    os.mkdir(os.path.join('dataset', "test", cl))
  except OSError:
    print (f"No se pudo crear folder test {cl}")
  else:
    print (f"Se creó folder test {cl}")
  



Se creó folder dataset
Se creó folder dataset
Se creó folder test Noticias
Se creó folder test Comercial


In [8]:
dataset = dict()

for index, cl in enumerate(classesFinal):
    grouped_data = df.groupby(by=["Clase"]).get_group(cl)["Nombre Final"].tolist()
    dataset[classesFinal[index]] = grouped_data

In [9]:
for k,v in dataset.items():
    print(f'Class: {k}, Length: {len(v)}')

    print(v)

Class: Noticias, Length: 3154
['2021_10_27_09_F60', '2021_10_27_09_F120', '2021_10_27_09_F180', '2021_10_27_09_F240', '2021_10_27_09_F300', '2021_10_27_09_F360', '2021_10_27_09_F420', '2021_10_27_09_F480', '2021_10_27_09_F540', '2021_10_27_09_F600', '2021_10_27_09_F660', '2021_10_27_09_F720', '2021_10_27_09_F780', '2021_10_27_09_F840', '2021_10_27_09_F900', '2021_10_27_09_F960', '2021_10_27_09_F1020', '2021_10_27_09_F1080', '2021_10_27_09_F1140', '2021_10_27_09_F1200', '2021_10_27_09_F1260', '2021_10_27_09_F5040', '2021_10_27_09_F5100', '2021_10_27_09_F5160', '2021_10_27_09_F5220', '2021_10_27_09_F5280', '2021_10_27_09_F5340', '2021_10_27_09_F5400', '2021_10_27_09_F5460', '2021_10_27_09_F5520', '2021_10_27_09_F5580', '2021_10_27_09_F5640', '2021_10_27_09_F5700', '2021_10_27_09_F5760', '2021_10_27_09_F5820', '2021_10_27_09_F5880', '2021_10_27_09_F5940', '2021_10_27_09_F6000', '2021_10_27_09_F6060', '2021_10_27_09_F6120', '2021_10_27_09_F6180', '2021_10_27_09_F6240', '2021_10_27_09_F6300

In [10]:
shutil.copy("/content/data/2021_10_27_09_F10320.jpg", "/content/dataset/test/Noticias/Hola.jpg")
shutil.copy("/content/data/2021_10_27_09_F10320.jpg", "/content/dataset/test/Comercial/Hola2.jpg")

'/content/dataset/test/Comercial/Hola2.jpg'

In [11]:
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/resnet18_finetuned.pth", "/content/resnet18_finetuned.pth")

'/content/resnet18_finetuned.pth'

In [12]:
pathDataset = 'dataset/'


test_dataset = torchvision.datasets.ImageFolder(pathDataset + 'test',
                                                    transform = transforms.Compose([ transforms.Resize(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

class_names = test_dataset.classes

device = ('cuda' if torch.cuda.is_available() else 'cpu')


def test_model_one(model, criterion):
  model.eval()
  running_loss = 0.0
  running_corrects = 0.0

  for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
            
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / len(test_dataset)
  epoch_acc = running_corrects / len(test_dataset)
  #print('Test Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))
  predscpu=preds.cpu()
  predscpuNumpy=predscpu.numpy()
  if predscpuNumpy[0]==1:
    prediccion="Noticias"
  else:
    prediccion="Comercial"

  return prediccion

def evaluate_imagen(imagedir,model,criterion):
  shutil.copy(imagedir, "/content/dataset/test/Noticias/Hola.jpg")
  shutil.copy(imagedir, "/content/dataset/test/Comercial/Hola2.jpg")

  return test_model_one(model, criterion)

model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 2)

model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load("resnet18_finetuned.pth"))
criterion = nn.CrossEntropyLoss()



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [13]:
evaluate_imagen("/content/data/2021_10_27_09_F5100.jpg",model_ft,criterion)

'Noticias'

In [14]:
evaluate_imagen("/content/drive/MyDrive/Taller Diplomado IA/Resultados/resultados_09/result_2021_11_26_22_frame_102600.png",model_ft,criterion)

'Noticias'

In [15]:
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/Comerciales_test.csv", "/content/Comerciales_test_v2.csv")

'/content/Comerciales_test_v2.csv'

In [16]:
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/data_test.zip", "/content/data_test.zip")
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/Comerciales_test.csv", "/content/Comerciales_test.csv")
!unzip data_test.zip

Archive:  data_test.zip
   creating: data_test/
  inflating: data_test/2021_11_01_13_F0.png  
  inflating: data_test/2021_11_01_13_F60.png  
  inflating: data_test/2021_11_01_13_F120.png  
  inflating: data_test/2021_11_01_13_F180.png  
  inflating: data_test/2021_11_01_13_F240.png  
  inflating: data_test/2021_11_01_13_F300.png  
  inflating: data_test/2021_11_01_13_F360.png  
  inflating: data_test/2021_11_01_13_F420.png  
  inflating: data_test/2021_11_01_13_F480.png  
  inflating: data_test/2021_11_01_13_F540.png  
  inflating: data_test/2021_11_01_13_F600.png  
  inflating: data_test/2021_11_01_13_F660.png  
  inflating: data_test/2021_11_01_13_F720.png  
  inflating: data_test/2021_11_01_13_F780.png  
  inflating: data_test/2021_11_01_13_F840.png  
  inflating: data_test/2021_11_01_13_F900.png  
  inflating: data_test/2021_11_01_13_F960.png  
  inflating: data_test/2021_11_01_13_F1020.png  
  inflating: data_test/2021_11_01_13_F1080.png  
  inflating: data_test/2021_11_01_13_F114

In [17]:
from os import listdir
from os.path import isfile, join
mypath = "/content/data_test"
ls_img = [f for f in listdir(mypath) if isfile(join(mypath, f))]
ls_img

['2021_11_01_13_F25380.png',
 '2021_11_01_13_F421080.png',
 '2021_11_01_13_F425040.png',
 '2021_11_01_13_F425100.png',
 '2021_11_01_13_F5760.png',
 '2021_11_01_13_F426960.png',
 '2021_11_01_13_F660.png',
 '2021_11_01_13_F421980.png',
 '2021_11_01_13_F25560.png',
 '2021_11_01_13_F3960.png',
 '2021_11_01_13_F427980.png',
 '2021_11_01_13_F25260.png',
 '2021_11_01_13_F421860.png',
 '2021_11_01_13_F424440.png',
 '2021_11_01_13_F422040.png',
 '2021_11_01_13_F23940.png',
 '2021_11_01_13_F22980.png',
 '2021_11_01_13_F5580.png',
 '2021_11_01_13_F423780.png',
 '2021_11_01_13_F2760.png',
 '2021_11_01_13_F423600.png',
 '2021_11_01_13_F21900.png',
 '2021_11_01_13_F420840.png',
 '2021_11_01_13_F425280.png',
 '2021_11_01_13_F2820.png',
 '2021_11_01_13_F600.png',
 '2021_11_01_13_F427620.png',
 '2021_11_01_13_F424200.png',
 '2021_11_01_13_F426120.png',
 '2021_11_01_13_F425820.png',
 '2021_11_01_13_F7860.png',
 '2021_11_01_13_F2520.png',
 '2021_11_01_13_F424740.png',
 '2021_11_01_13_F6660.png',
 '2021_1

In [18]:
compare = []
img_list = []
for image in ls_img:
  result = evaluate_imagen("/content/data_test/" + image,model_ft,criterion)
  compare.append(result)
  img_list.append(image[:-4])

In [19]:
compare

['Noticias',
 'Noticias',
 'Comercial',
 'Comercial',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Comercial',
 'Noticias',
 'Comercial',
 'Comercial',
 'Noticias',
 'Noticias',
 'Comercial',
 'Comercial',
 'Comercial',
 'Comercial',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Comercial',
 'Comercial',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Noticias',
 'Comercial',
 'Noticias',
 'Comercial',
 'Comercial',
 'Noticias',
 

In [20]:
df_test = pd.read_csv('Comerciales_test.csv', header=0, sep=",")
df_red = pd.DataFrame()
df_red['result_red'] = compare
df_red['Nombre Final'] = img_list
df_test = pd.merge(df_test, df_red, how='left', on='Nombre Final')
df_test.head()

,Nombre Final,Clase,result_red
0,2021_11_01_13_F0,Noticias,Noticias
1,2021_11_01_13_F60,Noticias,Noticias
2,2021_11_01_13_F120,Noticias,Noticias
3,2021_11_01_13_F180,Noticias,Noticias
4,2021_11_01_13_F240,Noticias,Noticias


In [21]:
print(len(df_test))

conditions = [
    (df_test['Clase'] == 'Noticias') & (df_test['result_red'] == 'Noticias'),
    (df_test['Clase'] == 'Comercial') & (df_test['result_red'] == 'Comercial'),
    (df_test['Clase'] == 'Noticias') & (df_test['result_red'] == 'Comercial'),
    (df_test['Clase'] == 'Comercial') & (df_test['result_red'] == 'Noticias'),
    ]
choices = [1, 1, 0, 0]
df_test['correct'] = np.select(conditions, choices,)
print(df_test['correct'].sum())

339
298


In [22]:
df_test.to_csv('resultados.csv')